In [1]:
# Note: this slide imports a bunch of stuff for the peano-indexed Vec example
# Don't forget to run me before starting the presentation!!!
from __future__ import annotations
from typing import Any, Generic, TypeVar
from dataclasses import dataclass
N,T = TypeVar("N", bound="Nat"), TypeVar("T")

import sys
ipython = get_ipython()

# If we want to show a type error, don't show the whole stacktrace
# (which is hard to interpret in slideshow-form)
def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    value.__cause__ = None  # suppress chained exceptions
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

# Liquid Types, Revisited
## Nathan Taylor (ntaylor@cs), Sam Thomas (sgt@cs)

### C S 395T: The Model Checking Paradigm, 24 April 2023

Type theory is widely adopted in the PL community, and with good reason: We have strong metatheoretic guarantees around:

* Soundness: A type system will always reject an invalid program
* Automation: A type system (may!) require minimal intervention to execute
* Subtyping: Safely substituting one type for another
* Termination: Type-theoretic algorithms will always terminate with an answer, usually in small-polynomial time


## A type theory paper in a model-checking class?

However...  In this class we've talked about things like propositions, proofs, counterproofs... some might say that this feel like a different world than the typed programming you know from languages like Java?

Under the Curry-Howard correspondence, we _know_[^1] that:

* a type _is_ a logical proposition;
* a well-typed program _is_ a proof of each of its propositions;
  * a type error _is_ a counterproof of one or more of its propositions

Type semantics relates a class of inputs  (all possible valid envioronments) to a class of outputs (all possible instantiations of a type) - feels like abstraction or “modulo”!!!

[^1] At least to a first-order approximation.  Some terms and conditions may apply; see in store for details.

## Do well-typed programs never go wrong?

The point of a type system should be that it rejects invalid programs, but: can you think of input to `avg()` that will still cause an exception to be thrown?

In [ ]:
def avg(xs: list[int]) -> int:
    """ Computes the average of a list of ints."""
    return sum(xs) // len(xs)

assert avg([1,2,3]) == 2
assert avg([1]) == 1

## Extending the expressivity of a type system

* Somebody in this room once said, "An abstraction is a constrained proof system"
    * a richer _constraint domain_ in our proof system lets us encode richer abstractions.
    * we may pay a cost for a richer _constraint domain_, however...

What sort of constraint domain would let us encode the type of a list that is provably nonempty (and thus avoiding the division by zero exception), while still maintaining soundness, automation, and termination?

---

Our goal: 
```python
def avg(xs: {TODO: proposition that rejects the empty list} ) -> int:
    return sum(xs) // len(xs)
avg([])      # should produce a type error
avg([1,2,3]) # should not produce a type error
```

## Non-solution 1: Keep the proof system; enrich the List type

A standard solution is to _index_ the List type with an additional type parameter which represents its length.

```python
# Here's our vector of N elements, all of type T.
@dataclass
class Vec(Generic[N,T]):
    ... # N and T are erased after typechecking
```

This means we need to teach _the type system_ about the natural numbers.

In [ ]:
# An encoding of the natural numbers, entirely in the type system
# (notice the S and Z classes have no fields!)
class Nat: pass
class Z(Nat): pass              # The Z type is zero
class S(Nat, Generic[N]): pass  # The S<N> type is the successor of some other nat N

In [ ]:
# Note: `Two` and `Four` are typedefs for some subtypes of Nat, not 
# program-level terms with values we can compute on at runtime!
Two: type  = S[S[Z]]
Four: type = S[S[Two]]

The two constructors `nil` and `cons` enforce the invariant that the typelevel value of `N` always conforms to the length of the list.

We did this without needing to make the type system more expressive, and the benefits we already get from it remain!

In [ ]:
# Here's our vector of N elements, all of type T.
@dataclass
class Vec(Generic[N,T]):
    l: list[T]
        
def nil() -> Vec[Z, T]: return Vec([])
def cons(t: T, l: Vec[N, T]) -> Vec[S[N], T]: return Vec([t] + l.l)

# The type system knows the lengths of these lists!
empty: Vec[Z, int] = nil()
one_three:  Vec[Two, int] = cons(1, cons(3, nil()))

```python
def avg(l: Vec[S[N], int]) -> int:
    """ Produces the average of a vector of ints, whose length
    is the successor of some natural number (i.e. greater than 0!)"""
    return sum(l.l) // len(l.l)

avg(42)    # E: "Literal[42]" is incompatible with "Vec[S[N@avg], int]"
avg(nil()) # E: TypeVar "N@Vec" is covariant; "Z" is incompatible with "S[N@avg]"
avg(cons(1, nil())) # ok, as we'd hope
```

✓ This implementation of `avg` statically rejects the empty list!  This was the point of the exercise.

✓ All the existing benefits of our existing type system (soundness, automation, subtyping, termination) apply here, because we haven't made the metatheory more complicated

✗ However, counterproofs are opaque and difficulty to map back to the original program error.

## Non-solution 2: enrich the proof system; keep the List type

Previously, `Vec[N,T]` took two type-level arguments.  We could not parameterize it on a program term of type `int` (if we could, we could have avoided redefining the natural numbers in the type system.)

A type system that _does_ allow this is called a _dependent type system_.  

```haskell
constant vec : Type u → ℕ → Type u

-- notice that we can write 0 and (n + 1) directly! 
constant empty : Π α : Type u, vec α 0
constant cons :
  Π (α : Type u) (n : ℕ), α → vec α n → vec α (n + 1)
```

The tradeoff is the one we expect: a richer type system lets us express more things, may require hand-written lemmas to help the proof system, losing the push-button automation that was so nice in the previous non-solution.

## Non-solution 3: just use a model checker

"At no point is `avg()` passed the empty list" is a temporal safety property; any number of off-the-shelf tools (SLAM, BLAST, Houdini, etc...) or techniques from class could be leveraged here.  



✓ We've seen that model checking techniques are sound, push-button in their automation, and can scale to non-trivial problems, just like a good typechecker.


✓ Since model checking explores program states, it's sensitive to program control flow, leading to a more precise abstraction than a type-based approach.

✗ However... as we saw from other presentations, reasoning about inductive data structures (lists, trees, etc...) typically requires quantifiers and/or custom logics.  With types, quantifications are handled via polymorphism and recursive definitions pose no difficulty.

## Unifying the non-solutions with _liquid types_

A _refinement type_ pairs an ordinary type (the _base type_) with a _refining predicate_:

\begin{equation}
\{v: int\; | \; 0 \leq v\; ∧\; v < n\} 
\end{equation}

This particular type refines its inhabitants to be between 0 and some in-scope program variable `n`.  `n` is a term, so a refinement type is a dependent type, too!  

$v$ is the distinguished _value variable_, referring to whatever term the type is typing.

### If predicates are conjunctions drawn from _SMT-decidable theories_, then we have a _logically-qualified (LiQuid) type system_.

```python
@solvent.check
def avg(xs: {len(V) > 0}) -> int
    return sum(xs) // len(xs)
avg(42) # Type-checking error
avg([]) # Type-checking error
```

In this talk, we will eludcidate the liquid type technique by way of our implementation of liquid types for Python (See https://github.com/dijkstracula/solvent)

Two classes of type meta-algorithms:

* Type _checking_: Verifies that a program fragment _annotated_ with types is consistent within the theory
* Type _reconstruction_: Given a program fragment _unannotated_ with types, infer a consistent set of types
  * generally the harder problem since it requires coming up with a consistent type, not just verifying it

Naturally, reconstruction is the problem this paper is interested in, setting the bar high!

## Reconstruction of a liquid type...

\begin{equation}
\{v: \text{<base type>} \;|\; \text{<predicate>} \wedge \text{<predicate>} \wedge \text{...} \} 
\end{equation}
    
Involves decomposing the problem into reconstructing the base type and the refinements: for each term to be typed, we perform:
    
1) Base type inference, using the venerable Hindley-Milner algorithm;
2) Liquid constraint generation, capturing subtyping relationships that must be satisfied;
3) Liquid constraint solving, finding a "good" conjunction of predicates that satisifies the constraints.
    
The key insight of the paper comes up at step 3!

## Which base type to reconstruct?

```python
def max(x: T1, y: T2) -> T3:
    if x > y:
        return x
    else:
        return y
```

We'll begin by assigning _abstract constraint variables_ for each of the program terms, and then _unifying_ those constraints.

Intuitively: 
* Because `x` and `y` are used in an arthmetic context, their types need to be `int`s (or subtypes of `int`).
* Because the return type `T3` is derived from `x` or `y`, it needs to be an `int` (or a superclass of `int`).

TODO: sammy please confirm your dyslexic pal nathan did not mess this up

A fascinating property of constraint-based typechecking is that we are guaranteed to get the most general type assignments - unification will never overfit to the constraint data.  So, not only do we get proofs for free, we get _the best_ proofs for free!

In [ ]:
from solvent.frontend import *

@infer_base
def max(x, y):
    if x > y:
        return x
    else:
        return y

## Which logical constraints to reconstruct?

```python
def max(x: int, y: int) -> int:
    if x > y:
        return x
    else:
        return y
```

Now that we have our base types figured out, we turn to infering refinement predicates.

What logical facts can we say about each particular int in this program?

## Which refinement predicate to reconstruct?

```python
def max(x: int, y: int) -> { V: int | TODO: erm...what predicate should be chosen }
    if x > y:
        return x
    else:
        return y
print(max(42, 99))
```

\begin{equation}
\{v: int \;|\; true\}  \text{...probably too underspecified!}
\end{equation} 

\begin{equation}
\begin{split}
\{v: int \;|\; v >= x\; \wedge \; v >= y\}\; & \text{...seems like a good choice!} \\
\{v: int \;|\; v = x\; \vee \; v = y\}\; & \text{...overfits a bit!}
\end{split}
\end{equation}

\begin{equation}
\{v: int \;|\; v = 99\} \text{... certainly way too overspecified!}
\end{equation}

## Which logical constraints to reconstruct?

```python
def max(x: {int | K1}, y: {int | K2}) -> {int | K3}:
    if x > y:
        return x
    else:
        return y
```

We begin by lifting our base types into refinement types.  These predicates will, just like with base type inference, begin as abstract constraints that we'll figure out by processing the function.

There are two kinds of ways that we can constrain $K_1$, $K_2$, and $K_3$:

* A _scope constraint_ is a constraint that depends on program terms that are in scope at the type definition point.
* A _flow constraint_ is a constraint that depends on how we might flow through the control flow graph. 

## What scope constraints do we have?

```python
def max(x: {int | K1}, y: {int | K2}) -> {int | K3}:
    if x > y:
        return x
    else:
        return y
```

Nothing's in scope for $K_1$ and $K_2$, so their typing contexts (e.g. to the left of the $\vdash$) are empty.

\begin{equation}
\begin{split}
& \; \vdash \; K_1 \\
& \; \vdash \; K_2
\end{split}
\end{equation}

By contrast, because `x` and `y` are in scope for the function body, their types can potentially affect $K_3$, so they appear in its context.

\begin{equation}
\begin{split}
x:\text{int}; \, y:\text{int} & \; \vdash \; K_3 \\
\end{split}
\end{equation}

## What _flow_ constraints do we have?

```python
def max(x: {int | K1}, y: {int | K2}) -> {int | K3}:
    if x > y:
        return x
    else:
        return y
```

As with before, $K_3$ is the interesting one, because its type depends on both arms of the `if` statement here.  $K_3$ is the _join_ of whatever types $K_1$ and $K_2$ turn out to be.

## What _flow_ constraints do we have?

```python
def max(x: {int | K1}, y: {int | K2}) -> {int | K3}:
    if x > y:
        return x # When x > y, we know that { int | V == x } is a subtype of K3!
    else:
        return y
```

\begin{equation}
\begin{split}
x:\text{int}; \, y:\text{int}; \, (x > y) & \; \vdash \; \{ int \;|\; V = x \} <: \{ int \;|\; K_3 \} \\
\\
\end{split}
\end{equation}

## What _flow_ constraints do we have?

```python
def max(x: {int | K1}, y: {int | K2}) -> {int | K3}:
    if x > y:
        return x 
    else:
        return y ## when !(x > y), we know that { int | V == y } is a subtype of K3!
```

\begin{equation}
\begin{split}
x:\text{int}; \, y:\text{int}; & \; (x > y) & \; \vdash \; \{ int \;|\; V = x \} <: \{ int \;|\; K_3 \} \\
x:\text{int}; \, y:\text{int}; & \; \neg(x > y) & \; \vdash \; \{ int \;|\; V = y \} <: \{ int \;|\; K_3 \}
\end{split}
\end{equation}

In [1]:
from solvent.frontend import *

@infer_constraints
def max(x, y):
    if x > y:
        return x
    else:
        return y

Function: max
  Constraints:
    x > y |- { int | V == x } <: { int | 't6 }
    -(x > y) |- { int | V == y } <: { int | 't6 }


By analyzing all possible control flows through the function, we've found path-dependent properties of the return value of `max()`:

\begin{equation}
\begin{split}
x:\text{int}; \, y:\text{int}; & \; (x > y) & \; \vdash \; \{ int \;|\; V = x \} <: K_3 \\
x:\text{int}; \, y:\text{int}; & \; \neg(x > y) & \; \vdash \; \{ int \;|\; V = y \} <: K_3
\end{split}
\end{equation}

We _could_ take our two subtypes and construct a disjunction out of them in order to come up with a type for $K_3$ - in fact, this was one of the types we proposed a moment ago!

\begin{equation}
\begin{split}
K_3 := \{ int \;|\; V = x \vee V = y \} \\
\end{split}
\end{equation}

This indeed satifies our constraints...

\begin{equation}
\{ int \;|\; \textbf{V = x} \} <: \{ int \;|\; \textbf{V = x} \vee V = y \} \\
\{ int \;|\; \textbf{V = y} \} <: \{ int \;|\; V = x \vee \textbf{V = y} \}
\end{equation}

However, while _technically correct_, this type isn't as general as we'd like.  

## Drawing from a fixed set of qualifiers

Because there are any number of possible refinement types, a liquid type system _fixes_ a "basis set" of logical qualifiers patterns that predicates can be generated from.  The paper's worked examples requires only a small subset of the full set:

\begin{equation}
\begin{split}
0 & \leq v & \\
\star & \leq v   \; & \text{for $\star$ : int}     \\
\star & \lt v     \; & \text{for $\star$ : int}    \\
v & \lt len(\star) \; & \text{for $\star$ : array} \\
\end{split}
\end{equation}

Recall that $v$ is the distinguished _value variable_, referring to whatever term the type is typing.

$\star$ is a special _placeholder_ that well-typed program terms can be substituted in for.

Something _not_ discussed in the paper is that these qualifiers can be overriden by the application programmer.

For instance, the liquid type `Vec` implementation in the paper artefact comes with an additional set of qualifiers, written by the implementer of the data structure, specific to that program:

```ocaml
(* a portion of ./postests/vec/len.hquals, simplified *)
qualif LVAR(v): len(v) [<, =, >] *
qualif LCONST(v): len(v) [<, =, >]  [0, 1]
qualif SETAPPEND(v): len(v) = (len(v) > * ? len(v) : * + 1)
qualif LSUM(v) : len(v) = [0, 1] + len( * ) + len( * )
qualif TOARR(v): len( * ) = Array.length v
```
In this way, both the type system implementer and the application programmer, can _bias_ the inferred types.  

## Concretizing qualifiers with scope constraints

We need to expand our qualifiers by substituting all our scope constraints for $\star$.  Any well-type program term in scope can replace a $\star$ 

\begin{equation}
\begin{split}
0 & \leq v; \;\;\;
\star & \leq v  \, \text{for $\star$ : int}; \;\;\;
v & \lt \star \, \text{for $\star$ : int};    \;\;\;
v & \lt len(\star) \, \text{for $\star$ : array} \;\;\;
\end{split}
\end{equation}

## Concretizing qualifiers with scope constraints

\begin{equation}
\begin{split}
0 & \leq v; \;\;\;
\star & \leq v  \, \text{for $\star$ : int}; \;\;\;
v & \lt \star \, \text{for $\star$ : int};    \;\;\;
v & \lt len(\star) \, \text{for $\star$ : array} \;\;\;
\end{split}
\end{equation}

\begin{equation}
\begin{split}
& \; \vdash \; K_1 \\
& \; \vdash \; K_2
\end{split}
\end{equation}

Because $x:K_1$ and $y:K_2$ have no scope constraints (nothing's in scope for them), we have nothing to substitute for $\star$, so only $ 0 \leq v $ remains.  This means that the only possible subtype of $K_1$ that we could synthesize will be of the form:

\begin{equation}
\begin{split}
\{ int \;|\; 0 & \leq v \} & \; <: K_1 \\
\{ int \;|\; 0 & \leq v \} & \; <: K_2 \\
\end{split}
\end{equation}


## Concretizing qualifiers with scope constraints

\begin{equation}
\begin{split}
0 & \leq v; \;\;\;
\star & \leq v  \, \text{for $\star$ : int}; \;\;\;
v & \lt \star \, \text{for $\star$ : int};    \;\;\;
v & \lt len(\star) \, \text{for $\star$ : array} \;\;\;
\end{split}
\end{equation}

\begin{equation}
\begin{split}
x:\text{int}; \, y:\text{int}; & \; (x > y) & \; \vdash \; \{ int \;|\; v = x \} <: K_3 \\
x:\text{int}; \, y:\text{int}; & \; \neg(x > y) & \; \vdash \; \{ int \;|\; v = y \} <: K_3
\end{split}
\end{equation}

Because $K_3$ give us more interesting constraints, because we generated scope constraints - `x` and `y` can be substituted in for $\star$.


In our example, we can use these qualifier templates to fill in an initial guess for $K_3$

\begin{equation}
\begin{split}
0 \leq v & & 
\star \leq v & &
\star \leq v & &
\star \lt v & &
\star \lt v \\
\{ int \;|\; v = x \} <: ( int \;|\: 0 \leq v & \; \wedge \;
                        & x \leq v & \; \wedge \;
                        & y \leq v & \; \wedge \;
                        & x \lt v & \; \wedge \;
                        & y \lt v) \\
\{ int \;|\; v = y \} <:( int \;|\: 0 \leq v & \; \wedge
                        & x \leq v & \; \wedge \;
                        & y \leq v & \; \wedge \;
                        & x \lt v & \; \wedge \;
                        & y \lt v) \\
\end{split}
\end{equation}



We have no idea whether these are _logically-consistent_ subtypes.  (in fact, there are outright contradictions in these!)

The final step is to _weaken_ these over-constrained types as minimally as possible, to have the most expressive consistent subtype.

## What's in a subtype?

At this point we're trying to generate "the best" refinement type for $K_3$.  How do we quantify which one is "the best"?

Remember an informal definition of subtyping, courtesy of Liskov: "if the supertype is valid in some typing context, then the subtype can be substituted for that term."  This is an implication!  So, we can express the subtyping relation as an implication, too!

\begin{equation}
\begin{split}
(x > y) & \; \vdash \; \{ int \;|\; V = x \} & \; <: & K_3 \\
\neg(x > y) & \; \vdash \; \{ int \;|\; V = y \} & \; <: & K_3 \\
\\
(x > y) & \; \wedge \; V = x & \implies & K_3 \\
\neg(x > y) & \; \wedge \; V = y & \implies & K_3 \\
\end{split}
\end{equation}

So, because we chose our refinement domain well, solving whether a subtyping relationship holds can be handed off to Z3.

## The type-weakening refinement loop

What should we do if Z3 comes back and tells us that the subtyping relationship we gave it was invalid?  _Refine and try again!_

Unsurprisingly, the solver will tell us that our initial subtyping relationship was incorrect.  This doesn't hold!

\begin{equation}
\begin{split}
\{ int \;|\; v = x \} <: \{ int \;|\: 0 \leq v & \; \wedge \;
                        & x \leq v & \; \wedge \;
                        & y \leq v & \; \wedge \;
                        & x \lt v & \; \wedge \;
                        & y \lt v \}
\end{split}
\end{equation}

To weaken the term, we throw out all the qualifiers that are not super-types of $\{int \; | \; v = x\}$

\begin{equation}
\begin{split}
\{ int \;|\; v = x \} <: \{ int \;|\: \textbf{0 $\leq$ v} & \; \wedge \;
                        & x \leq v & \; \wedge \;
                        & y \leq v & \; \wedge \;
                        & x \lt v & \; \wedge \;
                        & y \lt v \}
\end{split}
\end{equation}

\begin{equation}
\begin{split}
(x > y) & \; \vdash \; & \{ int \;|\; v = x \; \} & <: & \{ int \;|\; \textbf{0 $\leq$ v} \; \} \\
(x > y) & \; \wedge \; & V = x & \implies & \textbf{0 $\leq$ v} \\
\end{split}
\end{equation}

\begin{equation}
\textbf{✗ not valid!}
\end{equation}

\begin{equation}
\begin{split}
\{ int \;|\; v = x \} <: \{ int \;|\: 0 \leq v & \; \wedge \;
                        & \textbf{x $\leq$ v} & \; \wedge \;
                        & y \leq v & \; \wedge \;
                        & x \lt v & \; \wedge \;
                        & y \lt v \}
\end{split}
\end{equation}

\begin{equation}
\begin{split}
(x > y) & \; \vdash \; & \{ int \;|\; v = x \; \} & <: & \{ int \;|\; \textbf{x $\leq$ v} \; \} \\
(x > y) & \; \wedge \; & V = x & \implies & \textbf{x $\leq$ v} \\
\end{split}
\end{equation}

\begin{equation}
\textit{✓ valid!}
\end{equation}

\begin{equation}
\begin{split}
\{ int \;|\; v = x \} <: \{ int \;|\: 0 \leq v & \; \wedge \;
                        & x \leq v & \; \wedge \;
                        & \textbf{y $\leq$ v} & \; \wedge \;
                        & x \lt v & \; \wedge \;
                        & y \lt v \}
\end{split}
\end{equation}


\begin{equation}
\begin{split}
(x > y) & \; \vdash \; & \{ int \;|\; v = x \; \} & <: & \{ int \;|\; \textbf{y $\leq$ v} \; \} \\
(x > y) & \; \wedge \; & V = x & \implies & \textbf{y $\leq$ v} \\
\end{split}
\end{equation}

\begin{equation}
\textit{✓ valid!}
\end{equation}

\begin{equation}
\begin{split}
\{ int \;|\; v = x \} <: \{ int \;|\: 0 \leq v & \; \wedge \;
                        & x \leq v & \; \wedge \;
                        & y \leq v & \; \wedge \;
                        & \textbf{x $\lt$ v} & \; \wedge \;
                        & y \lt v \}
\end{split}
\end{equation}


\begin{equation}
\begin{split}
(x > y) & \; \vdash \; & \{ int \;|\; v = x \; \} & <: & \{ int \;|\; \textbf{x $\lt $ v} \; \} \\
(x > y) & \; \wedge \; & V = x & \implies & \textbf{x $\lt $ v} \\
\end{split}
\end{equation}

\begin{equation}
\textbf{✗ not valid!}
\end{equation}

\begin{equation}
\begin{split}
\{ int \;|\; v = x \} <: \{ int \;|\: 0 \leq v & \; \wedge \;
                        & x \leq v & \; \wedge \;
                        & y \leq v & \; \wedge \;
                        & x \lt v & \; \wedge \;
                        & \textbf{y $\lt$ v} \}
\end{split}
\end{equation}

\begin{equation}
\begin{split}
(x > y) & \; \vdash \; & \{ int \;|\; v = x \; \} & <: & \{ int \;|\; \textbf{y $\lt$ v} \; \} \\
(x > y) & \; \wedge \; & V = x & \implies & \textbf{y $\lt$ v} \\
\end{split}
\end{equation}

\begin{equation}
\textit{✓ valid!}
\end{equation}

We now have a shorter guess for $K_3$! Let's check this against our constraints again, to see how we are doing.

\begin{equation}
\begin{split}
✓ \; & \{ int \;|\; v = x \} <: ( int \;|\: x \leq v & \; \wedge \;
                        & y \leq v & \; \wedge \;
                        & y \lt v) \\
✗ \; & \{ int \;|\; v = y \} <:( int \;|\: x \leq v & \; \wedge \;
                        & y \leq v & \; \wedge \;
                        & \textbf{y $\lt$ v}) \\
\end{split}
\end{equation}

Our second constraint is still not valid.


We cam remove continue removing failing constraints, and then checking again until we have found a solution.

# The end-to-end typechecking process

In [4]:
from solvent.frontend import *

@infer_full
def max(x, y):
    if x > y:
        return x
    else:
        return y

Function: max
  Constraints:
    bool = bool
    'x19 = 'y20
    'x19 = 'ifRes23
    'x19 = int
    'y20 = int
    'ifRes23 = 'guess21
  Ununified type: ('x19, 'y20) -> { 'guess21 | 't22 }
  HM Solution:
    'guess21 := int
    'ifRes23 := int
    'y20 := 'ifRes23
    'x19 := 'y20
SubTyping constraints
x > y |- { int | V == x } <: { int | 't24 }
-(x > y) |- { int | V == y } <: { int | 't24 }
 |- { int | 't24 } <: { int | 't22 }
[x = 1, y = 0, .v = 1]
Weakening x > y |- { int | V == x } <: { int | 't24 }
 trying 0 <= V: [x = -1, y = -2, .v = -1]
 trying x <= V: valid
 trying y <= V: valid
 trying V < x: [x = 1, y = 0, .v = 1]
 trying V < y: [x = 1, y = 0, .v = 1]
sol:
  guess21: int
  ifRes23: int
  y20: 'ifRes23
  x19: 'y20
  t22: 0 <= V and x <= V and y <= V and V < x and V < y
  t24: x <= V and y <= V
[y = 0, x = 0, .v = 0]
Weakening  |- { int | 't24 } <: { int | 't22 }
 trying 0 <= V: [.v = -1, y = -1, x = -1]
 trying x <= V: valid
 trying y <= V: valid
 trying V < x: [y = 0, x = 0,